In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
from tensorflow.keras.callbacks import TensorBoard

In [2]:
from sentence_transformers import SentenceTransformer

# Replace 'sentence-transformers/bert-base-nli-mean-tokens' with your desired model name
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
model = SentenceTransformer(model_name)


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
df = pd.read_parquet('clean_data.pqt')

In [4]:
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,aww matches background colour seemingly stuck ...,0,0,0,0,0,0
2,000113f07ec002fd,hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,make real suggestions improvement wondered sec...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,second time asking view completely contradicts...,0,0,0,0,0,0
159567,ffea4adeee384e90,ashamed horrible thing put talk page 128 61 19 93,0,0,0,0,0,0
159568,ffee36eab5c267c9,spitzer umm theres actual article prostitution...,0,0,0,0,0,0
159569,fff125370e4aaaf3,looks like actually put speedy first version d...,0,0,0,0,0,0


In [5]:
embeddings= model.encode(df['comment_text'].tolist())
df['emb'] = embeddings.tolist()

In [ ]:
embeddings.shape

(400, 768)

In [ ]:
max_len = df['emb'].apply(len).max()
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Conv1D, MaxPooling1D, Flatten
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
sequences = np.array(df['emb'].tolist())
sequences = sequences.reshape(400, 1, 768)
print("Shape of sequences before reshaping:", sequences.shape)

Shape of sequences before reshaping: (400, 1, 768)


In [ ]:
max_words = 1300  # Number of embeddings per sample
dmsn = 768     # Dimensionality of each embedding


In [ ]:


padded_sequences = pad_sequences(sequences, maxlen=max_words, padding='post', truncating='post')


In [ ]:
X = np.array(padded_sequences)
y = df.drop(['id','comment_text', 'emb'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=77)

In [ ]:
X_train.shape

(280, 1300, 768)

In [11]:
# dmsn = len(df['emb'].iloc[0])

# print("Dimensionality of the embeddings:", dmsn)

Dimensionality of the embeddings: 768


In [35]:
# # Reshape X_train and X_test to fit the RNN
# X_train = X_train.reshape(X_train.shape[0], max_words, dmsn)
# X_test = X_test.reshape(X_test.shape[0], max_words, dmsn)

ValueError: cannot reshape array of size 364000 into shape (280,1300,768)

In [ ]:
def history_plots(redux, metric):
    valid = "val_"+metric
    plt.plot(redux.history[metric], label=metric)
    plt.plot(redux.history[valid], label=valid)
    plt.legend()
    plt.show()

In [ ]:
filename= "ST_LSTM"
tb = TensorBoard(log_dir= "logs\\{}".format(filename))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Precision, Recall

i = Input(shape=(max_words, dmsn))
x = LSTM(128)(i)
x = Dense(64, activation='relu')(x)
x = Dense(6, activation='softmax')(x)
lstm_model2 = Model(i, x) 


In [ ]:
lstm_model2.compile(loss='categorical_crossentropy', optimizer= Adam(learning_rate=.000025), metrics=['accuracy', Recall(name= 'recall'), Precision(name='precision')])
r =lstm_model2.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test),callbacks=[tb])

Epoch 1/10
9/9 [==============================] - 6s 173ms/step - loss: 0.3520 - accuracy: 0.9929 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4330 - val_accuracy: 0.9917 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/10
9/9 [==============================] - 1s 174ms/step - loss: 0.3520 - accuracy: 0.9929 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4330 - val_accuracy: 0.9917 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/10
9/9 [==============================] - 1s 156ms/step - loss: 0.3520 - accuracy: 0.9929 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4330 - val_accuracy: 0.9917 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 1s 139ms/step - loss: 0.3519 - accuracy: 0.9929 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4330 - val_accuracy: 0.9917 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 5/10
9/9 [==============================] 

In [ ]:
history_plots(r,'accracy')

In [ ]:
history_plots(r,'loss')

In [ ]:
history_plots(r,'precision')

In [ ]:
history_plots(r,'recall')

In [ ]:
def f1_derive(hist=r,prc='precision',rcl='recall'):
    vp= 'val_'+prc
    v_re= 'val_'+rcl
    CH_hist= pd.DataFrame(hist.history)
    CH_hist['f1'] =(2*CH_hist[prc]*CH_hist[rcl])/(CH_hist[prc]+CH_hist[rcl])
    CH_hist['val_f1'] =(2*CH_hist[vp]*CH_hist[vp])/(CH_hist[vp]+CH_hist[vp])
    CH_hist.fillna(0,inplace=True)

    plt.plot(CH_hist['f1'], label='f1')
    plt.plot(CH_hist['val_f1'], label='val_f1')
    plt.legend()
    plt.show()

In [ ]:
f1_derive()

In [ ]:
filename= "ST_CNN"
tb = TensorBoard(log_dir= "logs\\{}".format(filename))

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Dense, Flatten

i = Input(shape=(max_words, dmsn))

# Conv1D layer
x = Conv1D(32, 3, activation='relu')(i)
x = MaxPooling1D(2)(x)

# Flatten layer
x = Flatten()(x)

# Dense layers
x = Dense(64, activation='relu')(x)
x = Dense(6, activation='softmax')(x)

# Create the Model
cnn_model = Model(i,x)


In [ ]:
cnn_model.compile(loss='categorical_crossentropy', optimizer= Adam(learning_rate=.000025), metrics=['accuracy', Recall(name= 'recall'), Precision(name='precision')])
g=cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
9/9 [==============================] - 4s 114ms/step - loss: 0.3516 - accuracy: 0.1821 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4327 - val_accuracy: 0.1583 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 2/10
9/9 [==============================] - 1s 66ms/step - loss: 0.3514 - accuracy: 0.2357 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4326 - val_accuracy: 0.2333 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 3/10
9/9 [==============================] - 1s 63ms/step - loss: 0.3511 - accuracy: 0.3286 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4323 - val_accuracy: 0.3417 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 4/10
9/9 [==============================] - 1s 59ms/step - loss: 0.3507 - accuracy: 0.4821 - recall: 0.0000e+00 - precision: 0.0000e+00 - val_loss: 0.4319 - val_accuracy: 0.5750 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00
Epoch 5/10
9/9 [==============================] - 1

In [ ]:
history_plots(g,'accuracy')

IndexError: tuple index out of range

In [ ]:
history_plots(g,'loss')

array([[0, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 2, ..., 0, 0, 0]])